# SLAVV Tutorial

This notebook demonstrates how to run the **Segmentation-Less, Automated, Vascular Vectorization (SLAVV)** pipeline on a 3D microscopy volume.

We will:
1. Load a 3D TIFF image.
2. Configure processing parameters.
3. Run the vectorization pipeline to extract vascular networks.
4. Visualize the results inline.
5. Export the network for 3D viewing.

In [ ]:
import sys
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

# Ensure project root is in path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.slavv.pipeline import SLAVVProcessor
from src.slavv.io_utils import load_tiff_volume, export_pipeline_results
from src.slavv.utils import validate_parameters

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 1. Load Data

Specify the path to your input TIFF file. If you don't have one, you can download `Image A` (20170809_txRed_chronic_tiled.tif) from the project resources.

In [ ]:
# Set your input path here
input_path = "../data/slavv_test_volume.tif"  # Example path, update this!

if not os.path.exists(input_path):
    print(f"Warning: File not found at {input_path}. Please update input_path.")
else:
    print(f"Loading {input_path}...")
    image = load_tiff_volume(input_path)
    print(f"Image loaded. Shape: {image.shape}, Dtype: {image.dtype}")

    # Visualize a middle slice
    plt.figure(figsize=(10, 10))
    plt.imshow(image[..., image.shape[2]//2], cmap='gray')
    plt.title("Middle Z-Slice")
    plt.colorbar(label='Intensity')
    plt.axis('off')
    plt.show()

## 2. Configure Parameters

Set the physical parameters of the microscope and the biological constraints of the vessels.

In [ ]:
params = {
    # Microscopy parameters (anisotropic voxel size)
    'microns_per_voxel': np.array([1.07, 1.07, 5.0]),
    
    # Optical parameters
    'approximating_PSF': True,
    'numerical_aperture': 0.95,
    'excitation_wavelength_in_microns': 2.6, # 1.3 * 2
    
    # Vessel geometry constraints
    'radius_of_smallest_vessel_in_microns': 1.5,
    'radius_of_largest_vessel_in_microns': 60.0,
    
    # Filter settings
    'gaussian_to_ideal_ratio': 0.5,
    'spherical_to_annular_ratio': 6.0/7.0,
    'scales_per_octave': 3.0,
    'energy_sign': -1.0,
    
    # Processing
    'max_voxels_per_node_energy': 1e9, # Disable chunking for small/medium volumes
    'energy_method': 'hessian'
}

# Validate parameters
validated_params = validate_parameters(params)
print("Parameters validated.")

## 3. Run Pipeline

Initialize the processor and run the full pipeline: `Energy -> Vertices -> Edges -> Network`.

In [ ]:
processor = SLAVVProcessor()

def progress_callback(frac, stage):
    print(f"Progress: {frac*100:.0f}% - {stage}")

output_dir = "tutorial_output"

try:
    results = processor.process_image(
        image, 
        validated_params, 
        progress_callback=progress_callback,
        checkpoint_dir=output_dir
    )
    print("Processing complete!")
except Exception as e:
    print(f"Processing failed: {e}")

## 4. Analyze Results

Inspect the extracted network statistics.

In [ ]:
if 'results' in locals():
    vertices = results['vertices']['positions']
    edges = results['edges']['traces']
    strands = results['network']['strands']
    
    print(f"Vertices found: {len(vertices)}")
    print(f"Edges traced: {len(edges)}")
    print(f"Network strands: {len(strands)}")
    
    # Simple radius histogram
    if len(results['vertices']['radii']) > 0:
        plt.figure(figsize=(10, 5))
        plt.hist(results['vertices']['radii'], bins=30, color='skyblue', edgecolor='black')
        plt.xlabel('Vessel Radius (microns)')
        plt.ylabel('Count')
        plt.title('Vessel Radius Distribution')
        plt.show()

## 5. Export for Visualization

Export the network to VMV format for viewing in Blender with the VessMorphoVis plugin.

In [ ]:
if 'results' in locals():
    from src.slavv.visualization import NetworkVisualizer
    
    visualizer = NetworkVisualizer()
    
    # Export to VMV
    vmv_path = visualizer.export_network_data(
        results, 
        os.path.join(output_dir, "network.vmv"), 
        format='vmv'
    )
    print(f"Exported VMV to: {vmv_path}")
    
    # Export pipeline results (CSV/JSON/MAT)
    export_pipeline_results(results, output_dir, base_name="tutorial")
    print(f"All results saved to {output_dir}/")